#<b><center>EDA review-Pennsylvania</b></center>

Importacion de librerías

In [1]:
from google.colab import drive
import pandas as pd
import seaborn as sns
import os
import json
from datetime import datetime
import matplotlib.pyplot as plt
from scipy.stats import zscore
import numpy as np
from google.cloud import storage
import pyarrow.parquet as pq
from google.colab import auth
auth.authenticate_user()

Montando carpeta drive.

In [2]:
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/datalake/reviews-estados/review-Pennsylvania
archivos = os.listdir()
df = pd.DataFrame()
for archivo in archivos:
    if archivo.endswith('.json'):  # Puedes ajustar la extensión según tus necesidades
        print('Cargando data ',archivo,'...')
        data = pd.read_json(archivo, lines=True)
        #df.append(data)
        df = pd.concat([df, data], axis=0)


/content/drive/MyDrive/datalake/reviews-estados/review-Pennsylvania
Cargando data  15.json ...
Cargando data  13.json ...
Cargando data  2.json ...
Cargando data  14.json ...
Cargando data  16.json ...
Cargando data  12.json ...
Cargando data  11.json ...
Cargando data  3.json ...
Cargando data  1.json ...
Cargando data  10.json ...
Cargando data  4.json ...
Cargando data  5.json ...
Cargando data  6.json ...
Cargando data  9.json ...
Cargando data  7.json ...
Cargando data  8.json ...


Se verifica el tipo de variable de cada columna en caso sea necesario cambiarlo.

In [4]:
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2400000 entries, 0 to 149999
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 164.8+ MB


,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.023762e+20,Bo Lottig,1605215315642,5,None,None,None,0x89cb403dccee5345:0x498030b8be38e6da
1,1.150407e+20,Jayleen Lawson,1604071628355,5,None,None,None,0x89cb403dccee5345:0x498030b8be38e6da
2,1.170220e+20,Richard Naugle,1614800202838,5,"Very knowledgeable, very helpful. Great place...",None,None,0x89c8c1a20671e5d5:0x1ca654f3761c121a
3,1.147554e+20,William Rosner,1619084783909,5,One of the best selections of vape juice and t...,None,None,0x89c8c1a20671e5d5:0x1ca654f3761c121a
4,1.014546e+20,Lee Eledge,1613063950115,5,Great and friendly place. Interesting discoun...,None,None,0x89c8c1a20671e5d5:0x1ca654f3761c121a


In [5]:
df['user_id'] = df['user_id'].astype(object)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2400000 entries, 0 to 149999
Data columns (total 8 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   user_id  object
 1   name     object
 2   time     int64 
 3   rating   int64 
 4   text     object
 5   pics     object
 6   resp     object
 7   gmap_id  object
dtypes: int64(2), object(6)
memory usage: 164.8+ MB


In [6]:
tiempos = []
for data in df['time']/1000:
  tiempos.append(datetime.utcfromtimestamp(data).strftime('%d/%m/%Y %H:%M:%S'))

df['time_normalizado'] = tiempos

In [7]:
df.head()

,user_id,name,time,rating,text,pics,resp,gmap_id,time_normalizado
0,102376163680081215488.0,Bo Lottig,1605215315642,5,None,None,None,0x89cb403dccee5345:0x498030b8be38e6da,12/11/2020 21:08:35
1,115040693285635391488.0,Jayleen Lawson,1604071628355,5,None,None,None,0x89cb403dccee5345:0x498030b8be38e6da,30/10/2020 15:27:08
2,117022003176866267136.0,Richard Naugle,1614800202838,5,"Very knowledgeable, very helpful. Great place...",None,None,0x89c8c1a20671e5d5:0x1ca654f3761c121a,03/03/2021 19:36:42
3,114755415607572250624.0,William Rosner,1619084783909,5,One of the best selections of vape juice and t...,None,None,0x89c8c1a20671e5d5:0x1ca654f3761c121a,22/04/2021 09:46:23
4,101454552174109032448.0,Lee Eledge,1613063950115,5,Great and friendly place. Interesting discoun...,None,None,0x89c8c1a20671e5d5:0x1ca654f3761c121a,11/02/2021 17:19:10


<b>Filtrado de columnas</b>

Se selecciona solo las columnas que pueden contener información relevante para el análisis.
- "pics" : no se realiza un análisis de imágenes
- "resp": son respuestas de los desarrolladores, generalmente de agredecimiento de los comentarios.

In [8]:
df_analisis = df[['user_id','name','time','time_normalizado','rating','text','gmap_id']]
df_analisis.head()

,user_id,name,time,time_normalizado,rating,text,gmap_id
0,102376163680081215488.0,Bo Lottig,1605215315642,12/11/2020 21:08:35,5,None,0x89cb403dccee5345:0x498030b8be38e6da
1,115040693285635391488.0,Jayleen Lawson,1604071628355,30/10/2020 15:27:08,5,None,0x89cb403dccee5345:0x498030b8be38e6da
2,117022003176866267136.0,Richard Naugle,1614800202838,03/03/2021 19:36:42,5,"Very knowledgeable, very helpful. Great place...",0x89c8c1a20671e5d5:0x1ca654f3761c121a
3,114755415607572250624.0,William Rosner,1619084783909,22/04/2021 09:46:23,5,One of the best selections of vape juice and t...,0x89c8c1a20671e5d5:0x1ca654f3761c121a
4,101454552174109032448.0,Lee Eledge,1613063950115,11/02/2021 17:19:10,5,Great and friendly place. Interesting discoun...,0x89c8c1a20671e5d5:0x1ca654f3761c121a


<b>Analizando las columnas numéricas</b>
- "time" se entiende como el número de registro.
- "rating" es la puntuación del usuario de 1 a 5, su variación se encuentra en ese rango

In [9]:
df_analisis.describe()

,time,rating
count,2.400000e+06,2.400000e+06
mean,1.553453e+12,4.345731e+00
std,4.188404e+10,1.091626e+00
min,6.626016e+11,1.000000e+00
25%,1.526765e+12,4.000000e+00
50%,1.555872e+12,5.000000e+00
75%,1.581711e+12,5.000000e+00
max,1.631133e+12,5.000000e+00


<b>Filtros complementarios</b>

Posibles duplicados y nulos. El uso de memoria se reduce a 60.8 Mb

In [ ]:
df_analisis.dropna(inplace=True)
df_analisis.drop_duplicates(inplace=True)
df_analisis.reset_index(drop=True,inplace=True)
df_analisis.info()
df_analisis.head(5)

<ipython-input-10-d87d968368bb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_analisis.dropna(inplace=True)


La media indica un valor de 4.34 puntos, lo que habla muy bien de los locales en Pennsylvania con 1.33 millones de reviews de respaldo.

In [ ]:
df_analisis.describe()

<b>Pairplot</b>

Como era de suponerse, la puntuación es discreta entre 1 y 5. En el gráfico "rating" vs "time" se observa que los primeros reviews fueron pocos y con una tendencia entre 3 o 4 puntos; sin embargo, el gráfico no ayuda a distinguir las siguientes puntuaciones.

In [ ]:
sns.pairplot(df_analisis)

Del pairplot se aprecian outliers de la columna 'time', por cual se realiza un análisis de la columna para eliminar valores atípicos

In [ ]:
ax = sns.boxplot( x ="time",data = df_analisis, linewidth = 2)

ax.set_title('Valores atípicos de tiempo de registro')
ax.set_xlabel('Tiempo de registro')

Se extraen los valores atípicos. Con 3 desviaciones estándar se aseguro solo descartar los valores atípicos.



In [ ]:
z_scores = zscore(df_analisis['time'])
abs_z_scores = np.abs(z_scores)

# Select the outliers using a threshold of 3
outliers = df_analisis[abs_z_scores > 3]
df_analisis = df_analisis[abs_z_scores <= 3]
outliers.head()
outliers.info()

La columna 'time' no tiene mayor uso que este análisis, para ahorrar memoria se descartará

In [ ]:
df_analisis.drop(columns=['time'],inplace=True)
df_analisis.drop_duplicates(inplace=True)
df_analisis.reset_index(drop=True,inplace=True)

In [ ]:
df_analisis.info()

In [ ]:
ratings_count = df_analisis[['rating','user_id']].groupby(by='rating', as_index=False).count().sort_values(by='rating', ascending=True)
ratings_count

In [ ]:
plt.figure(figsize=(10,4), dpi=100)
sns.barplot(data=ratings_count, x='rating', y='user_id')
plt.xlabel('Calificación')
plt.ylabel('N° Calificaciones')
plt.title("Calificaciones del Estado Pennsylvania")

plt.show()

Dividiendo en dataframes más pequeños y guardando lo obtenido:



In [ ]:
users = df_analisis[['user_id','name']]
users.drop_duplicates(inplace=True)
users.reset_index(drop=True,inplace=True)
users.info()

In [ ]:
ratings = df_analisis[['user_id','time_normalizado','gmap_id','rating']]
ratings.drop_duplicates(inplace=True)
ratings.reset_index(drop=True,inplace=True)
ratings.info()

In [ ]:
reviews = df_analisis[['user_id','time_normalizado','gmap_id','text']]
reviews.drop_duplicates(inplace=True)
reviews.reset_index(drop=True,inplace=True)
reviews.info()

In [ ]:
users.to_parquet('users.parquet')
ratings.to_parquet('ratings.parquet')
reviews.to_parquet('reviews.parquet')

Guardado en datalake de Google Storage:

In [ ]:
project_id = 'deep-wave-407518'
bucket_name = 'yelp-and-maps-data-processed'


In [ ]:
%cd /content/drive/MyDrive/datalake/reviews-estados/review-Pennsylvania

In [ ]:
# Google Storage user.parquet
# Configura el cliente de almacenamiento
client = storage.Client(project=project_id)

# Obtén el bucket
bucket = client.get_bucket(bucket_name)

# Define la ruta del archivo Parquet en Google Cloud Storage
gcs_path = 'Maps/reviews-estados/review-Pennsylvania/users.parquet'.format(bucket_name)

# Sube el archivo Parquet a Google Cloud Storage
blob = bucket.blob(gcs_path)
blob.upload_from_filename('users.parquet')


In [ ]:
# Google Storage user.parquet
# Configura el cliente de almacenamiento
client = storage.Client(project=project_id)

# Obtén el bucket
bucket = client.get_bucket(bucket_name)

# Define la ruta del archivo Parquet en Google Cloud Storage
gcs_path = 'Maps/reviews-estados/review-Pennsylvania/ratings.parquet'.format(bucket_name)

# Sube el archivo Parquet a Google Cloud Storage
blob = bucket.blob(gcs_path)
blob.upload_from_filename('ratings.parquet')

In [ ]:
# Google Storage user.parquet
# Configura el cliente de almacenamiento
client = storage.Client(project=project_id)

# Obtén el bucket
bucket = client.get_bucket(bucket_name)

# Define la ruta del archivo Parquet en Google Cloud Storage
gcs_path = 'Maps/reviews-estados/review-Pennsylvania/reviews.parquet'.format(bucket_name)

# Sube el archivo Parquet a Google Cloud Storage
blob = bucket.blob(gcs_path)
blob.upload_from_filename('reviews.parquet')